In [1]:
import os
from dotenv import load_dotenv

from src.config import Source, ChunkerConf, LLMConf, EmbedderConf, KnowledgeGraphConfig
from src.graph.knowledge_graph import KnowledgeGraph
from src.ingestion.embedder import ChunkEmbedder

import networkx as nx

env = load_dotenv('config.env')

## Graph Data Science
Goal of this notebook is to write some functions to enrich the graph with clustering algos and centrality measures

In [2]:
kg_config = KnowledgeGraphConfig(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name='vector'
)

embedder_conf = EmbedderConf(
    type=os.getenv("EMBEDDINGS_TYPE"),
    model=os.getenv("EMBEDDINGS_MODEL_NAME"),
)

embedder = ChunkEmbedder(conf=embedder_conf)

kg = KnowledgeGraph(conf=kg_config,embeddings_model=embedder.embeddings)

In [ ]:
kg.number_of_nodes

In [4]:
G = kg.get_digraph()

In [ ]:
G.nodes

In [ ]:
G.nodes.data()

In [ ]:
list(G.nodes())[0]

In [ ]:
G.edges

Use the nodes and edges and cluster them using Graph algos from networkx.

## Community Detection: Leiden VS Louvain
Both Leiden and Louvain are community detection algorithms for graph clustering, but Leiden improves upon Louvain in several key ways.

1️⃣ Louvain Algorithm
The Louvain method (Blondel et al., 2008) is a fast, greedy algorithm for modularity optimization.

🔹 How It Works  
1. Node Merging: Each node starts as its own community.
2. Modularity Gain: Nodes join neighboring communities to maximize modularity.
3. Aggregation: Communities are collapsed into "super-nodes," and steps 1–2 repeat.
4. Stops when modularity no longer improves.  

✅ Pros  
✔ Fast & scalable (works on large graphs)  
✔ Good modularity scores  
✔ Widely used in network science  

❌ Cons  
❌ Produces non-optimal partitions (sometimes "bad" communities)  
❌ Nodes can get stuck in local optima (suboptimal clustering)  
❌ Does not guarantee connected communities  


2️⃣ Leiden Algorithm
The Leiden method (Traag et al., 2019) is an improved version of Louvain, fixing many of its issues.

🔹 How It Works (Improved Louvain)   
1. Refines Louvain's modularity optimization
2. Ensures well-connected communities
3. Refines partitions to avoid "bad" community structures  

✅ Pros  
✔ More stable partitions (less randomness)  
✔ Guarantees well-connected communities  
✔ Avoids Louvain's local optima issue  
✔ Works better on directed & weighted graphs  

❌ Cons  
❌ Slightly slower than Louvain  
❌ Still dependent on resolution parameters  


| **Feature**                    | **Louvain**            | **Leiden**               |
|---------------------------------|------------------------|--------------------------|
| **Speed**                       | Faster                | Slightly slower          |
| **Partition Quality**           | Can be unstable       | More refined & stable    |
| **Community Connectivity**      | May have disconnected clusters | Always well-connected |
| **Avoids Local Optima?**        | No                    | Yes                      |
| **Better for Weighted Graphs?** | No                    | Yes  

## Modularity 
Modularity is a metric that measures the strength of division of a network into communities (clusters). It compares:  
✔ The actual number of edges inside communities  
✔ The expected number of edges if randomly placed  

Higher modularity (closer to 1) → Strong, well-separated communities
Lower modularity (closer to 0 or negative) → Weak or no community structure

$`Q=\frac{1}{2m} \sum_{ij} \left( A_{ij} - \frac{k_i k_j}{2m} \right) \delta(c_i, c_j)`$

where:  
- $`A_{ij}`$ = adjacency matrix (1 if edge exists, 0 otherwise)  
- $`( k_i)`$ , $`(k_j)`$ = degrees of nodes \( i \) and \( j \)  
- $`(m)`$ = total number of edges in the graph  
- $`(\delta(c_i, c_j))`$ = 1 if nodes \( i \) and \( j \) belong to the same community, 0 otherwise  

**🔹 Interpretation:**  
- \( Q \) **close to 1** → Strong, well-defined communities  
- \( Q \) **close to 0 or negative** → Weak or no community structure  

In [26]:
import community
from typing import Tuple
from logging import getLogger

logger = getLogger(__name__)

def detect_louvain_communities(G: nx.DiGraph, return_modularity:bool=True) -> nx.DiGraph | Tuple[nx.DiGraph, float]:
    
    G_undirected = G.to_undirected()

    partition = community.best_partition(G_undirected)  # Louvain method

    nx.set_node_attributes(G, partition, "community_louvain")  # Store communities in node attributes

    if not return_modularity:

        return G
    
    else: 
        modularity = community.modularity(partition, G_undirected)

        logger.info(f"Modularity based on Louvain communities: {modularity}")

        return G, modularity

In [ ]:
G, modularity = detect_louvain_communities(G)

print(f"Modularity based on Louvain communities: {modularity}")

for node, data in list(G.nodes(data=True))[:5]:
    print(f"Node {node} → Community {data['community_louvain']}")

In [30]:
import igraph as ig
import leidenalg

def detect_leiden_communities(G: nx.DiGraph, return_modularity:bool=True) -> nx.DiGraph | Tuple[nx.DiGraph, float]:
    """Convert nx.DiGraph to igraph and run Leiden algorithm"""
    
    # Convert networkx to igraph
    mapping = {node: i for i, node in enumerate(G.nodes())}  # Node mapping
    reverse_mapping = {i: node for node, i in mapping.items()}
    
    # Create igraph graph
    ig_G = ig.Graph(directed=True)
    ig_G.add_vertices(len(G.nodes()))
    ig_G.add_edges([(mapping[u], mapping[v]) for u, v in G.edges()])
    
    partition = leidenalg.find_partition(ig_G, leidenalg.ModularityVertexPartition)

    # Assign community labels back to NetworkX
    for i, comm in enumerate(partition):
        for node in comm:
            G.nodes[reverse_mapping[node]]["community_leiden"] = i 
    
    if not return_modularity:
        return G
    
    else:
        modularity = partition.modularity

        logger.info(f"Modularity based on Leiden communities: {modularity}")

        return G, modularity



In [ ]:
G, modularity = detect_leiden_communities(G)

print(f"Modularity based on Leiden communities: {modularity}")

for node, data in list(G.nodes(data=True))[:5]:
    print(f"Node {node} → Community {data['community_leiden']}")

In [ ]:
# TODO add this as a graph-wide property -> do it for both Leiden and Louvain
def store_modularity_in_neo4j(modularity, driver):
    """Save modularity score as a graph-wide property"""
    with driver.session() as session:
        session.run("MERGE (m:GraphMetric {name: 'modularity'}) SET m.value = $modularity", 
                    modularity=modularity)

store_modularity_in_neo4j(modularity, self._driver)

## Centrality measures
1️⃣ PageRank (Best for Directed Graphs!)
* Measures importance based on incoming edges
* Works great for web networks, citations, or social graphs
* Can highlight authoritative nodes inside a community

2️⃣ Betweenness Centrality (Good for Finding Bridges)
* Measures how often a node acts as a bridge
* High values indicate nodes connecting different communities
* Useful for detecting influencers or bottlenecks

3️⃣ Closeness Centrality (Who is the "Core" of a Community?)
* Measures how close a node is to others
* Useful for finding well-connected members inside a cluster

In [32]:
def compute_centralities(G: nx.DiGraph | nx.Graph) -> nx.DiGraph | nx.Graph:
    """Compute PageRank, Betweenness and Closeness Centralities and store them as metadata in the graph"""
    
    pr = nx.pagerank(G, alpha=0.85)
    bc = nx.betweenness_centrality(G)
    cc = nx.closeness_centrality(G)

    nx.set_node_attributes(G, pr, "pagerank")
    nx.set_node_attributes(G, bc, "betweenness")
    nx.set_node_attributes(G, cc, "closeness")
    
    return G

In [ ]:
G = compute_centralities(G)

for node, data in list(G.nodes(data=True))[:5]:
    print(f"Node {node} → PageRank {data['pagerank']}")
    print(f"Node {node} → Betweenness {data['betweenness']}")
    print(f"Node {node} → Closeness {data['closeness']}")


In [36]:
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.cm as cm

In [37]:
pagerank_values = nx.get_node_attributes(G, "pagerank")

min_pr = min(pagerank_values.values())
max_pr = max(pagerank_values.values())

node_size = [5000 * (pagerank_values[node] - min_pr) / (max_pr - min_pr) for node in G.nodes]

cmap = cm.plasma  # You can choose other colormaps like 'viridis', 'inferno', etc.
node_color = [pagerank_values[node] for node in G.nodes]  # Map PageRank to color



In [ ]:
# Plotting the graph
plt.figure(figsize=(12, 12))

# Layout of the graph (e.g., spring layout)
pos = nx.spring_layout(G, seed=42)  # You can use other layouts (e.g., circular, spectral)

# Draw the graph with custom node size and color based on PageRank
nx.draw(
    G,
    pos,
    with_labels=False,
    node_size=node_size,       # Node size based on PageRank
    node_color=node_color,     # Node color based on PageRank
    cmap=cmap,                 # Colormap
    font_size=10,              # Font size for node labels
    font_weight='bold',        # Font weight for node labels
    edge_color='gray',         # Edge color
    alpha=0.7                 # Transparency of the node
)

# Add color bar to show the PageRank scale
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_pr, vmax=max_pr))
sm.set_array([])  # Empty array for the color bar

# Display the plot
plt.title("DiGraph Visualized with PageRank")
plt.show()

In [ ]:
bt_values = nx.get_node_attributes(G, "betweenness")

min_bt = min(bt_values.values())
max_bt = max(bt_values.values())

node_size = [5000 * (bt_values[node] - min_bt) / (max_bt - min_bt) for node in G.nodes]

cmap = cm.viridis
node_color = [bt_values[node] for node in G.nodes]  

plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, seed=42) 
nx.draw(
    G,
    pos,
    with_labels=False,
    node_size=node_size,       
    node_color=node_color,     
    cmap=cmap,                 
    font_size=10,              
    font_weight='bold',        
    edge_color='gray',         
    alpha=0.7                 
)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_bt, vmax=max_bt))
sm.set_array([]) 

# Display the plot
plt.title("DiGraph Visualized with Betweenness Centrality")
plt.show()

In [ ]:
ct_values = nx.get_node_attributes(G, "closeness")

min_ct = min(ct_values.values())
max_ct = max(ct_values.values())

node_size = [5000 * (ct_values[node] - min_ct) / (max_ct - min_ct) for node in G.nodes]

cmap = cm.inferno
node_color = [ct_values[node] for node in G.nodes]  

plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, seed=42) 
nx.draw(
    G,
    pos,
    with_labels=False,
    node_size=node_size,       
    node_color=node_color,     
    cmap=cmap,                 
    font_size=10,              
    font_weight='bold',        
    edge_color='gray',         
    alpha=0.7                 
)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_ct, vmax=max_ct))
sm.set_array([]) 

# Display the plot
plt.title("DiGraph Visualized with Closeness Centrality")
plt.show()

## Code to Update Neo4j

In [ ]:
def update_neo4j(G, driver):
    """Update Neo4j nodes with Leiden/Louvain community and centrality scores"""
    with driver.session() as session:
        for node, data in G.nodes(data=True):
            query = """
            MATCH (n) WHERE elementId(n) = $node_id
            SET n.community = $community,
                n.pagerank = $pagerank,
                n.betweenness = $betweenness
            """
            session.run(query, 
                        node_id=node, 
                        community=int(data.get("community", -1)), 
                        pagerank=float(data.get("pagerank", 0.0)), 
                        betweenness=float(data.get("betweenness", 0.0)))

# Connect to Neo4j and update the database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
update_neo4j(G, driver)
driver.close()